In [6]:
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [7]:
ROOT = Path('/home/jun0/Projects/emart24')
sys.path.append(str(ROOT))
print(sys.path)

['/home/jun0/Projects/emart24/emart24_codes/RJY', '/home/jun0/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles', '/home/jun0/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles/lib/python', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python38.zip', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8/lib-dynload', '', '/home/jun0/.local/lib/python3.8/site-packages', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8/site-packages', '/home/jun0/.local/lib/python3.8/site-packages/IPython/extensions', '/home/jun0/.ipython', '/home/jun0/Projects/emart24', '/home/jun0/Projects/emart24']


# 상품마스터

In [5]:
start_row = re.compile('^\d+\|')
i = 0
with open(ROOT / 'data/마스터+정보+및+데이터+정의서/02.설명이추가된 상품마스터_20210325.txt', 'r', encoding='utf-16') as f:
    cols = f.readline().replace('\n', '').split('|')
    rows = []
    past_line = ''
    while True:
        curr_line = f.readline()
        if not curr_line:
            break
        
        # 가장먼저 코드값으로 시작하는지 확인
        flag = start_row.search(curr_line)

        # 시작하는 줄이면 이전 past 저장하고 past로 넘김
        if flag:
            if past_line:
                row = past_line.replace('\n', '').split('|')
                rows.append(row)
            past_line = curr_line
            continue
        
        # curr 보고 하자 있으면 past에 붙이기
        if curr_line.count('|') < 45:
            past_line += ' ' + curr_line
        
df_desc = pd.DataFrame(rows, columns=cols)
df_desc.shape

(49299, 46)

In [6]:
wrong_float = df_desc.apply(lambda row: ('.' in row['점포발주입수수량']) or ('.' in row['공급가']) or ('.' in row['매가']) or ('.' in row['점포유통기한일수']), 1)
df_desc[wrong_float][['PLU_CD', '점포발주입수수량', '공급가', '매가', '점포유통기한일수']]

,PLU_CD,점포발주입수수량,공급가,매가,점포유통기한일수
45111,1500000101435,1,34999.8,50000,1
45112,4580395731629,1,10000.1,15000,1
45113,4580395731643,1,10000.1,15000,1
45242,8809112545261,1,10000.1,15000,1
45244,8809063770101,1,6499.9,9500,1
45245,8809112546145,1,4999.5,7500,548


In [8]:
df_desc.to_csv(ROOT / 'data/prep/02.설명이추가된 상품마스터_20210325.csv', index=False, sep='|')

In [ ]:
df_desc = pd.read_csv('./data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|')

In [15]:
df_desc.iloc[:2, :15]

,PLU_CD,상품명,대분류명,중분류명,소분류명,카테고리코드,상품속성코드,상품속성명,판매가능여부,시작일자,종료일자,사용여부,상온저온유형코드,상온저온유형명,시즌상품유형코드
0,9414453003319,KGB 크랜베리맛,자점,자점,자점,1151551055,401,일반,예,20140101,20190827,아니오,0,상온,01
1,9331275008295,레돈도)카푸치노125g,과자/빵,과자행사,비스켓행사(빼빼로),2092112004,401,일반,예,20200918,99991231,예,0,상온,01


In [16]:
df_desc.iloc[:2, 15:30]

,시즌상품유형명,행사상품유형코드,행사상품유형명,특별상품유형코드,특별상품유형명,낱개상품코드,신상품주기코드,신상품주기명,신상품시작일자,신상품종료일자,점포발주여부,점포발주가능일자,발주권장유형코드,발주권장유형명,반품유형코드
0,일반,01,일반,00,기본,,,,,,0,20140831,2,선택,0
1,일반,01,일반,00,기본,,04,4주,20200917,20201015,1,20200925,2,선택,2


In [17]:
df_desc.iloc[:2, 30:]

,반품유형명,점포발주입수수량,공급가,매가,점포유통기한일수,TRANL_CD,LT,MON_YN,TUE_YN,WED_YN,THU_YN,FRI_YN,SAT_YN,SUN_YN,GDS_DSCRT,GDS_SLPNT_DSCRT
0,불가,1,2400,3500,0,010221,1,0,0,0,0,0,0,0,,
1,한도,1,1700,2700,150,010092,1,1,1,1,1,1,1,0,1. 20년 빼빼로데이 기획상품 2. 틴 케이스에 웨이퍼롤이 담겨 있는 상품,1. 틴 케이스에 웨이퍼롤이 담겨 있는 상품


In [58]:
# 마스터 데이터
masters = {}
for f in sorted(Path('./data/이마트24_마스터데이터').glob('*.txt')):
    masters[f.name[3:-13]] = pd.read_csv(f, encoding="utf-16", sep='|', low_memory=False, dtype=str)
    print(f)
    print(masters[f.name[3:-13]].shape)

data/이마트24_마스터데이터/02.상품마스터_20210305.txt
(48981, 44)
data/이마트24_마스터데이터/03.행사마스터_20210305.txt
(142741, 32)
data/이마트24_마스터데이터/04.상품가격마스터_20210305.txt
(335736, 10)


In [61]:
imgs = set([f.name[:-4] for f in Path('./data/상품이미지+파일/').glob('*')])
cds = set(masters['상품마스터']['PLU_CD'].tolist())

In [65]:
len(imgs), len(cds)

(27835, 48981)

In [64]:
len(imgs & cds)

25683

# 거래데이터

In [35]:
# 거래데이터
emart24 = Path('./data/emart24Data')
f = emart24 / '001_TRAN/IFAI_001_TRAN_00115.txt'
# f = emart24 / '007_DAYQTY/IFAI_007_DAYQTY_00115.txt'
# f = emart24 / '008_STRPUR/IFAI_008_STRPUR_00115.txt'
# f = emart24 / '009_STRORD/IFAI_009_STRORD_00115.txt'
# f = emart24 / 'IFAI_100_RCMQTY_03193.txt'
temp = pd.read_csv(f, sep='|', low_memory=False)
temp

,DT,ORG_CD,PLU_CD,GDS_NM,LCLSS_CD,LCLSS_NM,MCLSS_CD,MCLSS_NM,SCLSS_CD,SCLSS_NM,...,DMT_OUT_QTY,DISU_QTY,STRUSE_QTY,STCK_BAL_DIFF_QTY,STCK_DIFF_QTY,SHIFT_STCK_QTY,WARIN_EDT_QTY,MANU_IN_QTY,MANU_GDS_QTY,END_QTY
0,20190321,A1600115,129,"POS감열지(표준사양,79*70mm,50롤)",114,소모품,144100,소모품,1002,영업소모품,...,0,0,0,0.0,0,0,0,0.0,0.0,11.0
1,20190912,A1600115,129,"POS감열지(표준사양,79*70mm,50롤)",114,소모품,144100,소모품,1002,영업소모품,...,0,0,0,0.0,0,0,0,0.0,0.0,12.0
2,20190916,A1600115,129,"POS감열지(표준사양,79*70mm,50롤)",114,소모품,144100,소모품,1002,영업소모품,...,0,0,0,0.0,0,0,0,0.0,0.0,13.0
3,20200625,A1600115,129,"POS감열지(표준사양,79*70mm,50롤)",114,소모품,144100,소모품,1002,영업소모품,...,0,0,0,0.0,0,0,0,0.0,0.0,15.0
4,20190506,A1600115,135,"쓰레기봉투검정(90X110, 박스:100장)",114,소모품,144100,소모품,1001,청소용품,...,0,0,0,0.0,0,0,0,0.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548952,20190610,A1600115,9791196067694,도서)12가지 인생의법칙,214,생활잡화,204200,서적,2001,일반서적,...,0,0,0,0.0,0,0,0,0.0,0.0,1.0
548953,20191112,A1600115,9791196067694,도서)12가지 인생의법칙,214,생활잡화,204200,서적,2001,일반서적,...,0,0,0,0.0,0,0,0,0.0,0.0,0.0
548954,20190612,A1600115,9791196316808,도서)공부머리 독서법,214,생활잡화,204200,서적,2001,일반서적,...,0,0,0,0.0,0,0,0,0.0,0.0,1.0
548955,20200629,A1600115,9791196831059,도서)코로나투자전쟁,214,생활잡화,204200,서적,2001,일반서적,...,0,0,0,0.0,0,0,0,0.0,0.0,1.0


In [61]:
temp.iloc[0, :]

DT                                   20190321
ORG_CD                               A1600115
PLU_CD                                    129
GDS_NM               POS감열지(표준사양,79*70mm,50롤)
LCLSS_CD                                  114
LCLSS_NM                                  소모품
MCLSS_CD                               144100
MCLSS_NM                                  소모품
SCLSS_CD                                 1002
SCLSS_NM                                영업소모품
TAX_TP                                      0
TAX_TP_NM                                  과세
GDS_COST                              37530.0
GDS_SAL_PRC                                 0
START_QTY                                11.0
WARIN_QTY                                   0
WARIN_BAL_QTY                               0
BUY_WARIN_QTY                               0
RTN_QTY                                     0
RTN_BAL_QTY                                 0
RTN_DISU_QTY                                0
SAL_QTY                           

In [58]:
temp.keys()

Index(['DT', 'ORG_CD', 'PLU_CD', 'GDS_NM', 'LCLSS_CD', 'LCLSS_NM', 'MCLSS_CD',
       'MCLSS_NM', 'SCLSS_CD', 'SCLSS_NM', 'TAX_TP', 'TAX_TP_NM', 'GDS_COST',
       'GDS_SAL_PRC', 'START_QTY', 'WARIN_QTY', 'WARIN_BAL_QTY',
       'BUY_WARIN_QTY', 'RTN_QTY', 'RTN_BAL_QTY', 'RTN_DISU_QTY', 'SAL_QTY',
       'SAL_RTN_QTY', 'SHIFT_WARIN_QTY', 'SHIFT_OUT_QTY', 'DMT_IN_QTY',
       'DMT_OUT_QTY', 'DISU_QTY', 'STRUSE_QTY', 'STCK_BAL_DIFF_QTY',
       'STCK_DIFF_QTY', 'SHIFT_STCK_QTY', 'WARIN_EDT_QTY', 'MANU_IN_QTY',
       'MANU_GDS_QTY', 'END_QTY'],
      dtype='object')

In [8]:
files = (ROOT / 'data/emart24Data/001_TRAN').glob('*.txt')
len(list(files))

226

In [11]:
# 거래데이터
# emart24 = ROOT / 'data/emart24Data'
# f = emart24 / '001_TRAN/IFAI_001_TRAN_00115.txt'
# f = emart24 / '007_DAYQTY/IFAI_007_DAYQTY_00115.txt'
# f = emart24 / '008_STRPUR/IFAI_008_STRPUR_00115.txt'
# f = emart24 / '009_STRORD/IFAI_009_STRORD_00115.txt'
# f = emart24 / 'IFAI_100_RCMQTY_03193.txt'
files = (ROOT / 'data/emart24Data/001_TRAN').glob('*.txt')
shapes = {}
transaction = pd.DataFrame()
for i, file in enumerate(files):
    transaction = pd.concat([transaction, pd.read_csv(file, sep='|', low_memory=False)], 0)
    if i == 1:
        break

# 파일 수, 데이터 건수
i + 1, transaction.shape

(2, (845626, 95))

In [14]:
transaction.shape

(845626, 95)

In [30]:
transaction['TRAN_NO'].unique().shape, transaction[['TRAN_YMD', 'TRAN_NO']].drop_duplicates().shape

((9999,), (416457, 2))

In [32]:
transaction[transaction['TRAN_NO'] == 1665]

,TRAN_YMD,COM_CD,STORE_CD,POS_NO,TRAN_NO,SYS_YMDHMS,TRAN_TYPE,TRAN_TYPE_NM,TRAN_KIND,TRAN_KIND_NM,...,EVT_CARD_AMT,SYS_YMD_B,BOT_GDS_CD,STAFF_DC_ID,STAFF_DC_QTY,STAFF_DC_AMT,APP_PAY_DC_ID,APP_PAY_DC_QTY,APP_PAY_DC_AMT,EVT_CD
0,20190101,1002,4890.0,1,1665,20190101000303,0,통상,0,판매,...,0.0,20181231.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
285,20190101,1002,4890.0,1,1665,20190101000303,0,통상,0,판매,...,0.0,20181231.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
18586,20190224,1002,4890.0,1,1665,20190224203115,0,통상,0,판매,...,0.0,20190224.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2.019011e+11
37139,20190409,1002,4890.0,1,1665,20190409090136,0,통상,0,판매,...,0.0,20190409.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
37140,20190409,1002,4890.0,1,1665,20190409090136,0,통상,0,판매,...,0.0,20190409.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425326,20201009,1002,2832.0,1,1665,20201009134755,0,통상,0,판매,...,0.0,20201009.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
442565,20201108,1002,2832.0,1,1665,20201108071436,1,환불,0,판매,...,0.0,20201108.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
442825,20201108,1002,2832.0,1,1665,20201108071436,1,환불,0,판매,...,0.0,20201108.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
460351,20201211,1002,2832.0,1,1665,20201211182514,0,통상,0,판매,...,0.0,20201211.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [20]:
transaction.iloc[:, :20].head(2)

,TRAN_YMD,COM_CD,STORE_CD,POS_NO,TRAN_NO,SYS_YMDHMS,TRAN_TYPE,TRAN_TYPE_NM,TRAN_KIND,TRAN_KIND_NM,PMOD_YN,STOP_YN,RFND_YN,OP_NO,ORG_TRAN_YMD,ORG_STORE_CD,ORG_POS_NO,ORG_TRAN_NO,ORG_RES_ID,APP_OP_NO
0,20190101,1002,4890.0,1,1665,20190101000303,0,통상,0,판매,0,0,NaN,4890,NaN,NaN,NaN,NaN,NaN,NaN
1,20190101,1002,4890.0,1,1666,20190101000352,0,통상,0,판매,0,0,NaN,4890,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
transaction.iloc[:, 20:40].head(2)

,CUST_CLS_ID,CUST_COUNT,ORDER_TYPE,FLOOR_CD,TABLE_NO,GSALE_AMT,GDC_AMT,NSALE_AMT,SCHARGE_AMT,DRAWER_CD,RECT_ISS_YMDHMS,REG_YMDHMS,PROC_ID,PROC_YMDHMS,PROC_CNT,PROC_PNO,LOAD_DTM,SYS_YMD,ITEM_SEQ,ASGN_CANCEL_YN
0,NaN,1,NaN,NaN,NaN,4700.0,0.0,4700,0,NaN,20190101000303,20190101000313,1.0,2019010100040950337,NaN,NaN,03-JAN-19 03.02.09.000000 AM,20181231,1,0.0
1,NaN,1,NaN,NaN,NaN,4500.0,0.0,4500,0,NaN,20190101000352,20190101000358,1.0,2019010100040950337,NaN,NaN,03-JAN-19 03.02.09.000000 AM,20181231,1,0.0


In [22]:
transaction.iloc[:, 40:60].head(2)

,ITEM_CD,PLU_CD,PLU_FLAG,CATE_ID,GRP_ID,SGRP_ID,BRAND_ID,ITEM_ID,ITEM_NM,SALE_QTY,SALE_UPRC,SALE_AMT,PRICE_CHG_ID,PRICE_CHG_UPRC,ITEM_INPUT_ID,SCAN_CD,BTL_SUM_AMT,BTL_PLU_CD,ITEM_TAX_ID,ITEM_TAX_VALUE
0,D118801094202606,8.801094e+12,0.0,107.0,117.0,1002.0,A1300001,0.0,스프라이트500ml,1.0,1800.0,1800.0,0.0,0.0,0.0,8.801094e+12,0.0,NaN,0.0,164.0
1,D110000088017624,8.801762e+07,0.0,113.0,151.0,1003.0,A1300001,0.0,P)팔리아멘트하이브리드5mg,1.0,4500.0,4500.0,0.0,0.0,0.0,8.801762e+07,0.0,NaN,0.0,409.0


In [23]:
transaction.iloc[:, 60:80].head(2)

,RECT_AMT,POS_DC_ID,POS_DC_RATE,POS_DC_AMT,EVT_DC_QTY,EVT_DC_AMT,CO_DC_ID,CO_DC_QTY,CO_DC_AMT,SALE_UNIT_QTY,DISUSE_FLAG,SCRAP_RES_ID,SERVICE_FLAG,PACK_FLAG,DBUYING_FLAG,REASON_ID,SALES_TYPE,REG_YMDHMS_B,PROC_ID_B,PROC_YMDHMS_B
0,1800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,NaN,1.0,2.019010e+13,0.0,NaN
1,4500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,NaN,1.0,2.019010e+13,0.0,NaN


In [24]:
transaction.iloc[:, 80:].head(2)

,PROC_CNT_B,PROC_PID_B,LOAD_DTM_B,EVT_CARD_TY,EVT_CARD_QTY,EVT_CARD_AMT,SYS_YMD_B,BOT_GDS_CD,STAFF_DC_ID,STAFF_DC_QTY,STAFF_DC_AMT,APP_PAY_DC_ID,APP_PAY_DC_QTY,APP_PAY_DC_AMT,EVT_CD
0,NaN,NaN,05-JAN-19 02.13.41.000000 PM,0.0,0.0,0.0,20181231.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,NaN,NaN,05-JAN-19 02.13.41.000000 PM,0.0,0.0,0.0,20181231.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
